<a href="https://colab.research.google.com/github/kiitaamuuraa/Asobiba/blob/main/MyFirstLXMERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LXMERT × object Refferal

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 3.2MB 36.4MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c39f941a21a7d0c1ac51522bd689a70b34652194697ba6759670949bc228c511
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, Dataset
from torch.utils.tensorboard import SummaryWriter

from transformers import LxmertTokenizer, LxmertModel

## トークナイザとモデルの定義

In [16]:
tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
model = LxmertModel.from_pretrained('unc-nlp/lxmert-base-uncased')

tensor([[[0.2955, 0.3331, 0.6159, 0.5383],
         [0.7964, 0.1018, 0.9522, 0.5114],
         [0.9440, 0.6369, 0.7073, 0.3295]]])

In [213]:
model

LxmertModel(
  (embeddings): LxmertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): LxmertEncoder(
    (visn_fc): LxmertVisualFeatureEncoder(
      (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
      (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (box_fc): Linear(in_features=4, out_features=768, bias=True)
      (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layer): ModuleList(
      (0): LxmertLayer(
        (attention): LxmertSelfAttentionLayer(
          (self): LxmertAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear

In [215]:
model.pooler.dense = nn.Linear(768, 2048)
model

LxmertModel(
  (embeddings): LxmertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): LxmertEncoder(
    (visn_fc): LxmertVisualFeatureEncoder(
      (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
      (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (box_fc): Linear(in_features=4, out_features=768, bias=True)
      (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layer): ModuleList(
      (0): LxmertLayer(
        (attention): LxmertSelfAttentionLayer(
          (self): LxmertAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear

## 入力（ダミー）の定義  
**注意:** 画像に関する情報を定義しないと、モデルはエラーを返す

In [50]:
sent = ['President Joe Biden sat down with ABC News George Stephanopoulos for a wide-ranging interview Tuesday in which he said his message to migrants was to not come to the border and that New York Gov.',
         'Andrew Cuomo should resign if allegations he committed sexual harassment are confirmed.',
        'And he said it would be "tough" to withdraw all American troops from Afghanistan by May 1, a deadline set out in a deal former President Donald Trump\'s administration made with the Taliban.']

inputs = tokenizer(sent, return_tensors="pt", padding=True, truncation=True)
print(inputs['input_ids'].shape)

# ダミーの画像特徴量を作成
# batch x num_images x dim
visual_feats = torch.randn([3, 64, 2048])
inputs['visual_feats'] = visual_feats

# ダミーの画像特徴量の座標作成
# batch x num_images x dim
visual_pos = torch.rand([3, 64, 4])
inputs['visual_pos'] = visual_pos

torch.Size([3, 43])


## 

In [229]:
outputs = model(**inputs)

In [217]:
# サイズ確認
for k in outputs.keys():
    print(k, outputs[k].shape)

language_output torch.Size([3, 43, 768])
vision_output torch.Size([3, 64, 768])
pooled_output torch.Size([3, 2048])


## ターゲットの画像のImage Feature から16個をランダムサンプル

In [218]:
import random

def get_idx(num_sample=16, all_options=63):
    idx = list()
    while len(idx) < num_sample:
        id = random.randint(0, all_options)
        if id not in idx:
            idx.append(id)
    return torch.tensor(idx)

l = list()
for i, b in enumerate(visual_feats):
    l.append(b[get_idx()])
    random_samples = torch.stack(l, dim=0)

random_samples.shape

torch.Size([3, 16, 2048])

## ランダムにラベルを作成

In [219]:
import numpy as np
labels = np.array([random.randint(0,15) for i in range(3)])

## 内積を取る

In [230]:
cross_modal_features = outputs['pooled_output'] # クロスモーダルの特徴量
print(cross_modal_features.shape)

# bmm用にリピート
cross_modal_features = cross_modal_features.unsqueeze(dim=1)
print(cross_modal_features.shape)

torch.Size([3, 2048])
torch.Size([3, 1, 2048])


In [231]:
batch_cos_sim = torch.bmm(cross_modal_features, random_samples.permute(0,2,1)).squeeze() # b(3) x target(16)

In [232]:
criterion = nn.CrossEntropyLoss()
loss = criterion(batch_cos_sim, torch.tensor(labels))
print(loss)

tensor(5.5077, grad_fn=<NllLossBackward>)


In [227]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [228]:
loss.backward()
optimizer.step()
optimizer.zero_grad()